# Load checkpoint and imports

In [1]:
import os
import sys
parent_dir = "/Midgard/home/farzantn/phd/Olfaction/MoLFormer_N2024"
sys.path.append(parent_dir) 
parent_dir="/Midgard/home/farzantn/mambaforge/envs/MolTran_CUDA11_cuda/lib/python3.8"
sys.path.append(parent_dir)

In [2]:
device_name='cuda'
from custom_utils.tokenizer.tokenizer import MolTranBertTokenizer
from constants import *
from utils.gs_lf import *
with open('../custom_utils/hparams.yaml', 'r') as f:
    config = Namespace(**yaml.safe_load(f))
from custom_utils.train_pubchem_light import LightningModule
from utils.util_alignment import *
import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 25
plt.rcParams["figure.figsize"] = (10,10)

/Midgard/home/farzantn/phd/Olfaction/MoLFormer_N2024/Notebooks


Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/Midgard/home/farzantn/mambaforge/envs/Mol/lib/python3.8/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


Mol


|# Loading Models

## Loading MolFormer Model

In [3]:
from finetune_multitask import MultitaskModel
tokenizer = MolTranBertTokenizer('../custom_utils/tokenizer/bert_vocab.txt')
# len(tokenizer.vocab)
def set_lm_frozen(ckpt):
    lm = LightningModule(config, tokenizer.vocab).load_from_checkpoint(ckpt, config=config, vocab=tokenizer.vocab)
    return lm
# 
# def set_lm_finetuned(ckpt):
#     
#     return lm
    
    

  # model = MultitaskModel(margs, tokenizer)
  #   else:
  #       print("# loaded pre-trained model from {args.seed_path}")
  #       model = MultitaskModel(margs, tokenizer).load_from_checkpoint(margs.seed_path, strict=False, config=margs, tokenizer=tokenizer, vocab=len(tokenizer.vocab))
        
ckpt_frozen = '../MoLformer_Pretrained/checkpoints/N-Step-Checkpoint_3_30000.ckpt'
lm_frozen = set_lm_frozen(ckpt=ckpt_frozen)

['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/Midgard/home/farzantn/phd/Olfaction/MoLFormer_N2024/Notebooks', '/Midgard/home/farzantn/mambaforge/envs/Mol/lib/python38.zip', '/Midgard/home/farzantn/mambaforge/envs/Mol/lib/python3.8', '/Midgard/home/farzantn/mambaforge/envs/Mol/lib/python3.8/lib-dynload', '', '/Midgard/home/farzantn/mambaforge/envs/Mol/lib/python3.8/site-packages', '/Midgard/home/farzantn/mambaforge/envs/Mol/lib/python3.8/site-packages/alvadesccliwrapper-1.0.8-py3.8.egg', '/Midgard/home/farzantn/phd/Olfaction/MoLFormer_N2024', '/Midgard/home/farzantn/mambaforge/envs/MolTran_CUDA11_cuda/lib/python3.8', '/Midgard/home/farzantn/phd/Olfaction/MoLFormer_N2024', '/Midgard/home/farzantn/mambaforge/envs/MolTran_CUDA11_cuda/lib/python3.8']
/Midgard/home/farzantn/phd/Olfaction/MoLFormer_N2024/Notebooks
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Usi

Global seed set to 12345


Using Rotation Embedding
/Midgard/home/farzantn/phd/Olfaction/MoLFormer_N2024/Notebooks
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding


Global seed set to 12345


Using Rotation Embedding
Using Rotation Embedding


In [4]:
# lm_frozen

# Extracting embedding from MoLFormer

## helper methods

In [5]:
def extract_molformer_finetuned(ds,input_file,cids,cids_subject,tasks):
    for j in range(29,31):
        print("j",j)
        ckpt = 'checkpoints_gs_lf/curated_GS_LF_merged_4983/'+str(j)+'/models_0000/checkpoint.ckpt'
        tokenizer = MolTranBertTokenizer('../custom_utils/tokenizer/bert_vocab.txt')
        print(len(tokenizer.vocab))
        # lm  = set_lm_finetuned(ckpt)
        config.num_tasks = len(gs_lf_tasks)
        config.dims = [768,768,768,1]
        lm = MultitaskModel(config, tokenizer).load_from_checkpoint(ckpt, config=config,tokenizer=tokenizer, vocab=len(tokenizer.vocab))
        
        X,y,X_layers,y_layers=extract_molformer_representations(lm, tokenizer, tasks, input_file, smiles_field)
        print("X",X.shape)
        # print("y",y.shape)
        
        
        # return X,y,X_layers,y_layers
        df_embeddings = convert_todf_molformer(X,cids,cids_subject,y)
        df_embeddings.to_csv(ds+'_molformerfinetuned_embeddings_13'+"_model_"+str(j)+'_Apr17.csv',     index=False)
        for i,X in enumerate(X_layers):
            print("i",i)
            df_embeddings = convert_todf_molformer(X_layers[i],cids,cids_subject,y_layers[i])
            df_embeddings.to_csv(ds+'_molformerfinetuned_embeddings_'+str(i)+"_model_"+str(j)+'_Apr17.csv', index=False)

In [6]:
def convert_todf_molformer(embeddings_dataset,cids,subjects=None,y=None):
    embeddings_dataset_copy = pd.DataFrame(embeddings_dataset)
    embeddings_dataset = pd.DataFrame()
    embeddings_dataset['embeddings'] = embeddings_dataset_copy.loc[:, 0:768].values.tolist()
    embeddings_dataset['CID'] = cids
    if subjects is not None:
        embeddings_dataset['subject'] = subjects
        
    if y is not None:
        y_dataset = pd.DataFrame(y)
        y_dataset['y'] = y_dataset.loc[:, 0:768].values.tolist()
    
        df = pd.concat([embeddings_dataset, y_dataset], axis=1)
        return df
    else:
        return embeddings_dataset

### Sagar

In [ ]:
input_file_sagar = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/mols_datasets/curated_sagar_subjects_nonaminus.csv' # or new downloaded file path
# '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/embeddings/molformer/gslf_molformer_embeddings_13_Apr17.csv'
smiles_field = 'nonStereoSMILES'
df_sagar_temp=pd.read_csv(input_file_sagar)
# sagar_tasks= df_sagar_temp.columns.to_list()[1:16]
cids_sagar= df_sagar_temp['cid'].values.tolist()
subjects_sagar= df_sagar_temp['subject'].values.tolist()


#### Frozen Molformer

In [ ]:
X_sagar,y_sagar,X_layers_sagar,y_layers_sagar=extract_molformer_representations(lm_frozen, tokenizer, sagar_tasks, input_file_sagar, smiles_field)
df_embeddings_sagar = convert_todf_molformer(X_sagar,cids_sagar,subjects_sagar,y_sagar)
# df_embeddings_sagar.to_csv('sagar_molformer_embeddings_13_Apr17.csv', index=False)
# for i,X in enumerate(X_layers_sagar):
#     print("i",i)
#     df_embeddings_sagar = convert_todf_molformer(X_layers_sagar[i],cids_sagar,subjects_sagar,c[i])
    # df_embeddings_sagar.to_csv('sagar_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)

In [ ]:
y_layers_sagar[0].shape

#### Fine-tuned MoLFormer

In [ ]:
extract_molformer_finetuned('sagar',input_file_sagar,cids_sagar,subjects_sagar,sagar_tasks)
# df_embeddings = convert_todf_molformer(X,cids,cids_subject,y)
        # df_embeddings.to_csv(ds+'_molformerfinetuned_embeddings_13'+"_model_"+str(j)+'Apr17.csv',     index=False)
        # for i,X in enumerate(X_layers):
        #     print("i",i)
        #     df_embeddings = convert_todf_molformer(X_layers[i],cids,cids_subject,y_layers[i])
        #     df_embeddings.to_csv(ds+'_molformerfinetuned_embeddings_'+str(i)+"_model_"+str(j)+'_Apr17.csv', index=False)

In [15]:
X_layers_sagar[0].shape

torch.Size([480, 768])

In [ ]:
# for j in range(1,31):
#     print("j",j)
#     ckpt = 'checkpoints_gs_lf/curated_GS_LF_merged_4983/'+str(j)+'/models_0000/last.ckpt'
#     tokenizer = MolTranBertTokenizer('../custom_utils/tokenizer/bert_vocab.txt')
#     print(len(tokenizer.vocab))
#     # lm  = set_lm_finetuned(ckpt)
#     config.num_tasks = len(gs_lf_tasks)
#     config.dims = [768,768,768,1]
#     lm = MultitaskModel(config, tokenizer).load_from_checkpoint(ckpt, config=config,tokenizer=tokenizer, vocab=len(tokenizer.vocab))
#     
#     X_sagar,y_sagar,X_layers_sagar,y_layers_sagar=extract_molformer_representations(lm, tokenizer, sagar_tasks, input_file_sagar, smiles_field)
#     df_embeddings_sagar = convert_todf_molformer(X_sagar,cids_sagar,subjects_sagar,y_sagar)
#     df_embeddings_sagar.to_csv('sagar_molformerfinetuned_embeddings_13_Apr17.csv', index=False)
#     for i,X in enumerate(X_layers_sagar):
#         print("i",i)
#         df_embeddings_sagar = convert_todf_molformer(X_layers_sagar[i],cids_sagar,subjects_sagar,y_layers_sagar[i])
#         df_embeddings_sagar.to_csv('sagar_molformerfinetuned_embeddings_'+str(i)+"_model_"+str(j)+'_Apr17.csv', index=False)

### Keller

In [7]:
input_file_keller = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/mols_datasets/curated_keller2016_nona.csv' # or new downloaded file path
smiles_field = 'nonStereoSMILES'
df_keller_temp=pd.read_csv(input_file_keller)
keller_tasks= df_keller_temp.columns.to_list()[5:]
cids_keller= df_keller_temp['CID'].values.tolist()
cids_subject_keller= df_keller_temp['Subject'].values.tolist()


#### Frozen Molformer

In [ ]:
X_keller,y_keller,X_layers_keller,y_layers_keller=extract_molformer_representations(lm_frozen, tokenizer, keller_tasks, input_file_keller, smiles_field)
df_embeddings_keller = convert_todf_molformer(X_keller,cids_keller,cids_subject_keller,y_keller)
df_embeddings_keller.to_csv('keller_molformer_embeddings_13_Apr17.csv', index=False)
for i,X in enumerate(X_layers_keller):
    print("i",i)
    df_embeddings_keller = convert_todf_molformer(X_layers_keller[i],cids_keller,cids_subject_keller,y_layers_keller[i])
    df_embeddings_keller.to_csv('keller_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)

#### Fine-tuned MoLFormer

In [8]:

extract_molformer_finetuned('keller',input_file_keller,cids_keller,cids_subject_keller,keller_tasks)

j 29
2362
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
dropout is 0.1
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
dropout is 0.1
X torch.Size([15134, 768])
i 0
i 1
i 2
i 3
i 4
i 5
i 6
i 7
i 8
i 9
i 10
i 11
j 30
2362
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embeddin

### Ravia

In [12]:
input_file_ravia = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/mols_datasets/ravia_molecules.csv'
df_ravia_temp=pd.read_csv(input_file_ravia)
smiles_field = 'nonStereoSMILES'
cids_ravia= df_ravia_temp['CID'].values.tolist()


In [ ]:
X_ravia,y_ravia,X_layers_ravia,y_layers_ravia=extract_molformer_representations(lm, tokenizer, [], input_file_ravia, smiles_field)
df_embeddings_ravia = convert_todf_molformer(X_ravia,cids_ravia,None,y_ravia)
df_embeddings_ravia.to_csv('ravia_molformer_embeddings_13_Apr17.csv', index=False)
for i,X in enumerate(X_layers_ravia):
    # print("i",i)
    df_embeddings_ravia = convert_todf_molformer(X_layers_ravia[i],cids_ravia,None,y_layers_ravia[i])
    df_embeddings_ravia.to_csv('ravia_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)

In [13]:
extract_molformer_finetuned('ravia',input_file_ravia,cids_ravia,None,[])

j 1
2362
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
dropout is 0.1
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
dropout is 0.1
X torch.Size([350, 768])
i 0
i 1
i 2
i 3
i 4
i 5
i 6
i 7
i 8
i 9
i 10
i 11
j 2
2362
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Us

### Snitz

In [14]:
input_file_snitz = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/mols_datasets/snitz_molecules.csv'
df_snitz_temp=pd.read_csv(input_file_snitz)
smiles_field = 'nonStereoSMILES'
cids_snitz= df_snitz_temp['CID'].values.tolist()

In [ ]:
X_snitz,y_snitz,X_layers_snitz,y_layers_snitz=extract_molformer_representations(lm, tokenizer, [], input_file_snitz, smiles_field)
df_embeddings_snitz = convert_todf_molformer(X_snitz,cids_snitz,None,y_snitz)
df_embeddings_snitz.to_csv('snitz_molformer_embeddings_13_Apr17.csv', index=False)
for i,X in enumerate(X_layers_snitz):
    print("i",i)
    df_embeddings_snitz = convert_todf_molformer(X_layers_snitz[i],cids_snitz,y_layers_snitz[i])
    df_embeddings_snitz.to_csv('snitz_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)

In [15]:
extract_molformer_finetuned('snitz',input_file_snitz,cids_snitz,None,[])

j 1
2362
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
dropout is 0.1
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
dropout is 0.1
X torch.Size([1320, 768])
i 0
i 1
i 2
i 3
i 4
i 5
i 6
i 7
i 8
i 9
i 10
i 11
j 2
2362
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
U

### GS-LF

In [16]:
input_file_gslf = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/mols_datasets/curated_GS_LF_merged_4983.csv' # or new downloaded file path
smiles_field = 'nonStereoSMILES'
df_gslf_temp=pd.read_csv(input_file_gslf)
gslf_tasks=df_gslf_temp.columns.to_list()[2:]
cids_gslf= df_gslf_temp.index.values.tolist()
# X_gslf,y_gslf,X_layers_gslf,y_layers_gslf=extract_molformer_representations(lm, tokenizer, gslf_tasks, input_file_gslf, smiles_field)

In [ ]:
# df_gslf_temp.index.values.tolist()

In [10]:
X_gslf, y_gslf, X_layers_gslf, y_layers_gslf = extract_molformer_representations(lm_frozen, tokenizer, gslf_tasks,
                                                                                 input_file_gslf, smiles_field)
df_embeddings_gslf = convert_todf_molformer(X_gslf,cids_gslf,None,y_gslf)
df_embeddings_gslf.to_csv('gslf_molformer_embeddings_13_Apr17.csv', index=False)
for i,X in enumerate(X_layers_gslf[:2]):
    # print("i",i)
    df_embeddings_gslf = convert_todf_molformer(X_layers_gslf[i],cids_gslf,None,y_layers_gslf[i])
    df_embeddings_gslf.to_csv('gslf_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)

ValueError: Expected a 1D array, got an array with shape (4983, 138)

In [28]:
X_gslf.shape

torch.Size([4983, 768])

In [17]:
extract_molformer_finetuned('gslf',input_file_gslf,cids_gslf,None,gslf_tasks)

j 1
2362
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
dropout is 0.1
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
dropout is 0.1
X torch.Size([4983, 768])
i 0
i 1
i 2
i 3
i 4
i 5
i 6
i 7
i 8
i 9
i 10
i 11
j 2
2362
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
U

In [22]:
# convert_todf_molformer(X_gslf,cids_gslf,None,y_gslf)
# embeddings_dataset = pd.DataFrame(X_gslf)
# embeddings_dataset['embeddings'] = embeddings_dataset.loc[:, 0:768].values.tolist()
# embeddings_dataset['CID'] = cids_gslf
# subjects = None
# if subjects is not None:
#     embeddings_dataset['subject'] = subjects
# embeddings_dataset
# 
# #     
# if y_gslf is not None:
#     y_dataset = pd.DataFrame(y_gslf)
#     y_dataset['y'] = y_dataset.loc[:, 0:768].values.tolist()
#     
# len(y_dataset['y'][0])
# # 
# df = pd.concat([embeddings_dataset, y_dataset], axis=1)
# len(df['y'][0])

138

## dravienks

In [18]:
input_file_dravienks_1 = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/tasks/dravnieks1985_applicability_1.csv' # or new  
# '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/embeddings/molformer/gslf_molformer_embeddings_13_Apr17.csv'
smiles_field = 'nonStereoSMILES'
df_dravienks_temp_1=pd.read_csv(input_file_dravienks_1)
dravienks_tasks_1= df_dravienks_temp_1.columns.to_list()[1:-9]
cids_dravienks= df_dravienks_temp_1['CID'].values.tolist()


In [12]:
X_dravienks,y_dravienks,X_layers_dravienks,y_layers_dravienks=extract_molformer_representations(lm_frozen, tokenizer, dravienks_tasks_1, input_file_dravienks_1, smiles_field)
df_embeddings_dravienks = convert_todf_molformer(X_dravienks,cids_dravienks,None,y_dravienks)
df_embeddings_dravienks.to_csv('dravienks1985App1_molformer_embeddings_13_Apr17.csv', index=False)
for i,X in enumerate(X_layers_dravienks):
    print("i",i)
    df_embeddings_dravienks = convert_todf_molformer(X_layers_dravienks[i],cids_dravienks,None,y_layers_dravienks[i])
    df_embeddings_dravienks.to_csv('dravienks1985App1_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)


In [19]:
extract_molformer_finetuned('dravienks1985App1',input_file_dravienks_1,cids_dravienks,None,dravienks_tasks_1)

j 1
2362
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
dropout is 0.1
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
dropout is 0.1
X torch.Size([144, 768])
i 0
i 1
i 2
i 3
i 4
i 5
i 6
i 7
i 8
i 9
i 10
i 11
j 2
2362
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Us

In [20]:
input_file_dravienks_2 = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/tasks/dravnieks1985_use_2.csv' # or new downloaded file path
# '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/embeddings/molformer/gslf_molformer_embeddings_13_Apr17.csv'
smiles_field = 'nonStereoSMILES'
df_dravienks_temp_2=pd.read_csv(input_file_dravienks_2)
dravienks_tasks_2= df_dravienks_temp_2.columns.to_list()[1:-9]
cids_dravienks= df_dravienks_temp_2['CID'].values.tolist()


In [10]:
X_dravienks,y_dravienks,X_layers_dravienks,y_layers_dravienks=extract_molformer_representations(lm_frozen, tokenizer, dravienks_tasks_2, input_file_dravienks_2, smiles_field)
df_embeddings_dravienks = convert_todf_molformer(X_dravienks,cids_dravienks,None,y_dravienks)
df_embeddings_dravienks.to_csv('dravienks1985Use2_molformer_embeddings_13_Apr17.csv', index=False)
for i,X in enumerate(X_layers_dravienks):
    print("i",i)
    df_embeddings_dravienks = convert_todf_molformer(X_layers_dravienks[i],cids_dravienks,None,y_layers_dravienks[i])
    df_embeddings_dravienks.to_csv('dravienks1985Use2_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)

NameError: name 'lm' is not defined

In [21]:
extract_molformer_finetuned('dravienks1985Use2',input_file_dravienks_2,cids_dravienks,None,dravienks_tasks_2)

j 1
2362
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
dropout is 0.1
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
dropout is 0.1
X torch.Size([144, 768])
i 0
i 1
i 2
i 3
i 4
i 5
i 6
i 7
i 8
i 9
i 10
i 11
j 2
2362
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Us